In [26]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

In [ ]:
leagueLinks = [
    "https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1",
    "https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1",
    "https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1",
    "https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1",
    "https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1",
]

listSquad = pd.DataFrame()
firstPartOfLink = "https://www.transfermarkt.com"
for link in leagueLinks:
    request = req.get(link, headers=headers)
    soup = bs(request.content, "html.parser")
    table = soup.select_one('table:contains("Club")')

    innerListSquad = pd.DataFrame(
        {
            "Squad": [
                (i["title"])
                for i in table.select("tbody td:nth-of-type(1) a:nth-of-type(1) img")
            ],
            "Squad Link": [
                (firstPartOfLink + i["href"])
                for i in table.select("tbody td:nth-of-type(2) a:nth-of-type(1)")
            ],
            "Mini Club Crest": [
                (i["src"])
                for i in table.select("tbody td:nth-of-type(1) a:nth-of-type(1) img")
            ],
        }
    )

    listSquad = pd.concat([listSquad, innerListSquad], axis=0)

def clubCrestBig(x):
    return x["Mini Club Crest"].replace("tiny", "head")

listSquad["Big Club Crest"] = listSquad.apply(clubCrestBig, axis=1)
listSquad

In [28]:
df0 = listSquad.sort_values(by="Squad", ascending=True)
df0 = df0.reset_index(drop=True)

In [ ]:
df0

In [ ]:
leagueTables = [
    "https://www.transfermarkt.com/premier-league/tabelle/wettbewerb/GB1/saison_id/2024",
    "https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1/saison_id/2024",
    "https://www.transfermarkt.com/ligue-1/tabelle/wettbewerb/FR1/saison_id/2024",
    "https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1/saison_id/2024",
    "https://www.transfermarkt.com/serie-a/tabelle/wettbewerb/IT1/saison_id/2024",
]

df1 = pd.DataFrame()
for link in leagueTables:
    re = req.get(link, headers=headers)
    so = bs(re.content, "html.parser")
    tb = so.select_one('table:contains("Goals")')
    df2 = pd.DataFrame(
        {
            "Team": [(i["title"]) for i in tb.select("tbody td:nth-of-type(2) a img")],
            "MP": [(i.text) for i in tb.select("tbody td:nth-of-type(4)")],
            "W": [(i.text) for i in tb.select("tbody td:nth-of-type(5)")],
            "D": [(i.text) for i in tb.select("tbody td:nth-of-type(6)")],
            "L": [(i.text) for i in tb.select("tbody td:nth-of-type(7)")],
            "Goals": [(i.text) for i in tb.select("tbody td:nth-of-type(8)")],
            "+/-": [(i.text) for i in tb.select("tbody td:nth-of-type(9)")],
            "Pts": [(i.text) for i in tb.select("tbody td:nth-of-type(10)")],
        }
    )

    df2["Pts/MP"] = df2["Pts"].astype(int) / df2["MP"].astype(int)
    df2["Pts/MP"] = df2["Pts/MP"].round(2)
    df1 = pd.concat([df1, df2], axis=0)
    df3 = df1.sort_values(by="Team", ascending=True)
    df4 = df3.reset_index(drop=True)
df4

In [ ]:
df5 = pd.concat([df4, df0], axis=1)
df5

In [34]:
df5 = df5.sort_values(by="Pts/MP", ascending=False)
df5["Rk"] = range(1, len(df5) + 1)
df5.insert(0, "Rk", df5.pop("Rk"))

In [ ]:
df5.set_index("Rk")

In [36]:
df5["TeamEqualToSquad"] = df5["Team"] == df5["Squad"]

if df5["TeamEqualToSquad"].all() == True:
    df = df5.drop(columns=["TeamEqualToSquad", "Squad"], axis=1)

In [ ]:
df

In [ ]:
listPlayer = pd.DataFrame()
for l in listSquad["Squad Link"]:
    r = req.get(
        l,
        headers=headers,
    )
    s = bs(r.content, "html.parser")
    t = s.select_one('table:contains("Market value")')

    innerListPlayer = pd.DataFrame(
        {
            "Player": [(i["title"]) for i in t.select("tbody td:nth-of-type(1) img")],
            "Position": [
                (i.text)
                for i in t.select("tbody td:nth-of-type(2) tr td:nth-of-type(1)")
            ][1::2],
            "Age": [(i.text) for i in t.select("tbody td:nth-of-type(3)")],
            "Market Value": [(i.text) for i in t.select("tbody .rechts.hauptlink")],
        }
    )

    clubs = s.find_all(class_="data-header__headline-wrapper data-header__headline-wrapper--oswald")
    clubName = [club.get_text(strip=True) for club in clubs]

    def addClubName(c):
        c = clubName[0]
        return c
    
    innerListPlayer["Club"] = innerListPlayer.apply(addClubName, axis=1)

    listPlayer = pd.concat([listPlayer, innerListPlayer], axis=0)

listPlayer["Age"] = listPlayer["Age"].str[-3:-1]
listPlayer["Position"] = listPlayer["Position"].replace("\n","", regex=True)
listPlayer["Position"] = listPlayer["Position"].str.strip()
listPlayer.index = range(0, len(listPlayer))

listPlayer

In [ ]:
marketValueOne = listPlayer[listPlayer["Market Value"].str.contains("k")]
listPlayer.drop(
    listPlayer[listPlayer["Market Value"].str.contains("k")].index,
    inplace=True,
)

marketValueSecond = marketValueOne[marketValueOne["Market Value"].str.contains("\xa0\xa0")]
marketValueOne.drop(
    marketValueOne[marketValueOne["Market Value"].str.contains("\xa0\xa0")].index,
    inplace=True,
)

In [ ]:
marketValueSeventh = pd.DataFrame()
marketValueSeventh["Market Value"] = marketValueOne["Market Value"].str[1:-1]
marketValueSeventh["Market Value"] = pd.to_numeric(marketValueSeventh["Market Value"])

marketValueSeventh

In [ ]:
valueList = []
for value in marketValueSeventh["Market Value"]:
    if value < 100 and value > 0:
        value = 0.1
        valueList.append(value)
    else:
        value = round((value / 1000), 1)
        valueList.append(value)

valueList

In [ ]:
marketValueSeventh["Market Value"] = valueList
marketValueSeventh

In [ ]:
marketValueEighth = pd.DataFrame()
marketValueEighth["Market Value"] = marketValueSecond["Market Value"].str[1:-3]
marketValueEighth["Market Value"] = pd.to_numeric(marketValueEighth["Market Value"])

marketValueEighth

In [ ]:
otherValueList = []
for v in marketValueEighth["Market Value"]:
    if v < 100 and v > 0:
        v = 0.1
        otherValueList.append(v)
    else:
        v = round((v / 1000), 1)
        otherValueList.append(v)

otherValueList

In [ ]:
marketValueEighth["Market Value"] = otherValueList
marketValueEighth

In [ ]:
marketValueOne.drop(columns=["Market Value"], inplace=True)
marketValueSecond.drop(columns=["Market Value"], inplace=True)

In [13]:
marketValueThird = pd.concat([marketValueOne, marketValueSeventh], axis=1)
marketValueNinth = pd.concat([marketValueSecond, marketValueEighth], axis=1)

marketValueTenth = pd.concat([marketValueThird, marketValueNinth], axis=0)

In [ ]:
marketValueTenth

In [ ]:
marketValueFourth = listPlayer[listPlayer["Market Value"].str.contains("m")]
listPlayer.drop(
    listPlayer[listPlayer["Market Value"].str.contains("m")].index,
    inplace=True,
)

marketValueFifth = marketValueFourth[marketValueFourth["Market Value"].str.contains("\xa0\xa0")]
marketValueFourth.drop(
    marketValueFourth[marketValueFourth["Market Value"].str.contains("\xa0\xa0")].index,
    inplace=True,
)

In [16]:
listPlayer["Market Value"] = 0.0
listPlayer.insert(4, "Market Value", listPlayer.pop("Market Value"))

In [ ]:
listPlayer

In [ ]:
marketValueSixth = pd.DataFrame()
marketValueSixth["Market Value"] = marketValueFourth["Market Value"].str[1:-1]
marketValueSixth["Market Value"] = pd.to_numeric(marketValueSixth["Market Value"])

marketValueSixth

In [ ]:
marketValueEleventh = pd.DataFrame()
marketValueEleventh["Market Value"] = marketValueFifth["Market Value"].str[1:-3]
marketValueEleventh["Market Value"] = pd.to_numeric(marketValueEleventh["Market Value"])

marketValueEleventh

In [ ]:
marketValueFourth.drop(columns=["Market Value"], inplace=True)
marketValueFifth.drop(columns=["Market Value"], inplace=True)

In [21]:
marketValueTwelfth = pd.concat([marketValueFourth, marketValueSixth], axis=1)
marketValueThirteenth = pd.concat([marketValueFifth, marketValueEleventh], axis=1)

marketValueFourteenth = pd.concat([marketValueTwelfth, marketValueThirteenth], axis=0)

In [ ]:
marketValueFourteenth

In [ ]:
finalListPlayer = pd.concat([marketValueTenth, marketValueFourteenth, listPlayer], axis=0)

finalListPlayer

In [ ]:
sortedListPlayer = finalListPlayer.sort_values(by="Club")

uniquePositions = sortedListPlayer["Position"].unique()

uniquePositions

In [ ]:
sortedListPlayer["Position"] = sortedListPlayer["Position"].replace("Goalkeeper", "GK")
sortedListPlayer["Position"] = sortedListPlayer["Position"].replace(["Centre-Back", "Left-Back", "Right-Back"], "DF")
sortedListPlayer["Position"] = sortedListPlayer["Position"].replace(["Defensive Midfield", "Central Midfield", "Right Midfield", "Left Midfield", "Attacking Midfield"], "MF")
sortedListPlayer["Position"] = sortedListPlayer["Position"].replace(["Centre-Forward", "Left Winger", "Right Winger", "Second Striker"], "FW")

sortedListPlayer.index = range(0, len(sortedListPlayer))

sortedListPlayer

In [ ]:
duplicatesByPlayer = sortedListPlayer[sortedListPlayer.duplicated(subset=["Player"], keep=False)]
duplicatesByPlayer